In [1]:
import numpy as np
from scipy import linalg as lg
import time

In [18]:
def count_layers(len_dim, factors):
    layers = []
    layer = []
    for n in range(len_dim):
        layer.append(factors[n].T @ factors[n])
    layers.append(layer)

    for n in range(1, len_dim - 1):
        layer = []
        for r in range(len_dim - n):
            layer.append(layers[n - 1][r] * layers[0][r + n])
        layers.append(layer)
    return layers

In [571]:
def count_W(len_dim, layers):
    W = []
    for row in range(len_dim):
        start_layer = row - 1
        end_layer = len_dim - row - 2
        if start_layer >= 0:
            W_col = layers[start_layer][0]
            if end_layer >= 0:
                W_col = W_col * layers[end_layer][-1]
        else:
            W_col = layers[end_layer][-1]

        W_row = [W_col]

        for col in range(row + 1, len_dim):
            start_layer = row - 1
            middle_layer = col - row - 2
            end_layer = len_dim - col - 2

            if start_layer >= 0:
                W_col = layers[start_layer][0]
                if middle_layer >= 0:
                    W_col = W_col * layers[middle_layer][row + 1]
                if end_layer >= 0:
                    W_col = W_col * layers[end_layer][-1]
            elif middle_layer >= 0:
                W_col = layers[middle_layer][row + 1]
                if end_layer >= 0:
                    W_col = W_col * layers[end_layer][-1]
            else:
                W_col = layers[end_layer][-1]


            W_row.append(W_col)

        W.append(W_row)

    return W

In [572]:
def count_JJ_d(len_dim, W, pos, jacob_size, dimensions, rank, factors, coef):
    JJ = np.zeros((jacob_size, jacob_size))
    for row in range(len_dim):
        BB = W[row][0]
        for r in range(dimensions[row]):
            now_pos = pos[row] + r * rank
            JJ[now_pos:now_pos+rank, now_pos:now_pos+rank] = BB

    for alpha in range(len_dim):
        now_alpha = pos[alpha]
        for i in range(alpha + 1, len_dim):
            now_i = pos[i]

            now_W = W[alpha][i - alpha]
            for betta in range(dimensions[alpha]):
                now_a_b = now_alpha + betta * rank
                for j in range(dimensions[i]):
                    now_i_j = now_i + j * rank
                    for gamma in range(rank):
                        now_a_b_g = now_a_b + gamma
                        for k in range(rank):
                            now_i_j_k = now_i_j + k
                            JJ[now_a_b_g, now_i_j_k] = now_W[gamma][k] * factors[i][j][gamma] * factors[alpha][betta][k]
                            JJ[now_i_j_k, now_a_b_g] = now_W[k][gamma] * factors[i][j][gamma] * factors[alpha][betta][k]


    for i in range(jacob_size):
        JJ[i][i] *= (1 + coef)

    return JJ

In [573]:
def count_J_A(len_dim, factors, rank, dimensions):
    J_start = [factors[0]]
    J_end = [factors[-1]]
    for i in range(1, len_dim - 1):
        J_start.append(lg.khatri_rao(J_start[i - 1], factors[i]))
        J_end.append(lg.khatri_rao(factors[-i - 1], J_end[i - 1]))
    A = lg.khatri_rao(J_start[-1], factors[len_dim - 1])
    A = np.sum(A, axis=1)

    J = np.zeros((np.prod(dimensions), np.sum(dimensions) * rank))
    for k in range(len_dim):
        pos = int(np.sum(dimensions[:k]) * rank)
        for i in range(dimensions[k]):
            E = np.zeros((dimensions[k], rank))
            E[i, :] = np.ones((1, rank))
            if k > 0:
                val = lg.khatri_rao(J_start[k - 1], E)
                if k < len_dim - 1:
                    val = lg.khatri_rao(val, J_end[len_dim - k - 2])
                J[:, pos + i * rank: pos + i * rank + rank] = val
            else:
                J[:, (pos + i * rank):(pos + i * rank + rank)] = lg.khatri_rao(E, J_end[len_dim - 2])

    return J, A

In [609]:
def Levenberg(tensor, rank, seed=42, tol = 10**(-8), n_iteration = 100):

    dimensions = tensor.shape
    len_dim = len(dimensions)

    jacob_size = int(np.sum(dimensions) * rank)

    rng = np.random.RandomState(seed)
    factors = np.array([rng.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)

    eps = 1.0
    it = 0

    pos = [0]
    for i in range(len_dim - 1):
        pos.append(pos[i] + dimensions[i] * rank)

    while eps > tol and it < n_iteration:

        layers = count_layers(len_dim, factors)
        W = count_W(len_dim, layers)

        JJ_d = count_JJ_d(len_dim, W, pos, jacob_size, dimensions, rank, factors, coef=1)
        J, A = count_J_A(len_dim, factors, rank, dimensions)
        delta = np.linalg.pinv(JJ_d) @ (J.T @ (A - tensor.reshape((1, -1))).T)

        for i in range(len_dim):
            position = int(np.sum(dimensions[:i]) * rank)
            for j in range(dimensions[i]):
                factors[i][j, :] -= (delta[position + j * rank:position + (j+1)*rank]).reshape(-1,)

        eps = np.linalg.matrix_norm(delta)

        it += 1

    return factors, it

In [610]:
sizes = np.array((10, 10, 10, 10))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            for l in range(sizes[3]):
                T[i, j, k, l] = np.sin(i + j + k + l)

In [611]:
rank = 4
factors, it = Levenberg(T, rank)

In [612]:
len_dim = len(sizes)
sweep_l = np.ones(rank).reshape(1, -1)

for j in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[j])

print(np.linalg.norm(np.sum(sweep_l, axis=1) - T.reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

6.584987198916426e-09
